In [112]:
import logic
import importlib
importlib.reload(logic)

<module 'logic' from 'c:\\workspace\\unimore-bioinfo-exam\\logic.py'>

In [113]:
#logic.set_all_seeds(42)

In [114]:
features, labels = logic.data_preparation("dataset_LUMINAL_A_B.csv", 40)

In [115]:
print(features.shape)
print(features)

print(labels.shape)
print(labels)

torch.Size([100, 40])
tensor([[ 8.1851e+01,  6.6550e+01,  1.2008e+00,  ...,  1.3154e-01,
          1.6021e-01, -5.0075e-02],
        [-5.4433e+00, -6.8498e-01,  2.3630e+00,  ..., -1.0433e-01,
          5.0505e-01,  3.7148e-02],
        [-7.9172e-01, -7.5666e-01, -2.9498e+00,  ...,  9.2409e-01,
          2.6931e+00,  3.3589e+00],
        ...,
        [-3.8432e+00,  4.5426e-01,  7.6024e+00,  ...,  1.7106e-01,
          4.7162e-01, -4.0518e-01],
        [-6.2338e-01, -8.1301e-01, -3.9153e+00,  ..., -2.8367e+00,
          1.0244e+00, -1.5090e+00],
        [-1.6019e+00,  8.8207e-01, -7.8587e+00,  ..., -8.1554e-01,
         -4.0350e+00, -2.8665e+00]])
torch.Size([100])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        

In [116]:
correlation = logic.compute_abs_pearson_correlation(features)

In [117]:
print(correlation)

tensor([[1.0000, 0.2675, 0.0732,  ..., 0.0839, 0.0563, 0.0127],
        [0.2675, 1.0000, 0.2096,  ..., 0.0888, 0.0213, 0.0133],
        [0.0732, 0.2096, 1.0000,  ..., 0.1192, 0.0770, 0.0778],
        ...,
        [0.0839, 0.0888, 0.1192,  ..., 1.0000, 0.0603, 0.0512],
        [0.0563, 0.0213, 0.0770,  ..., 0.0603, 1.0000, 0.0474],
        [0.0127, 0.0133, 0.0778,  ..., 0.0512, 0.0474, 1.0000]])


In [118]:
edges = logic.create_sparse_edges_with_threshold(correlation, 0.4)

In [119]:
print(edges)

tensor([[ 0,  1,  1,  1,  1,  2,  3,  3,  4,  4,  4,  4,  5,  6,  6,  6,  6,  6,
          6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  9, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 10, 11, 11, 11, 12, 13, 13, 14, 14, 14, 14, 14, 15,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 20, 21, 21, 21,
         21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 23, 23, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26,
         26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 28, 28, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 31, 31, 31,
         32, 32, 32, 32, 32, 32, 32, 32, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35,
         35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 38, 38, 38, 38,
         38, 38, 38, 40, 40, 42, 43, 43, 43, 43, 44, 45, 45, 45, 45, 45, 45, 45,
         45, 47, 47, 47, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 51, 51, 51,
         52, 53, 53, 53, 55,

In [120]:
number_of_points = features.shape[0]
train_mask, test_mask = logic.prepare_dataset_masks(number_of_points)

In [121]:
model = logic.GCN(input_channels=features.shape[1], hidden_channels=8, output_channels=2)
logic.train_gnn(model, 100, features, labels, edges, train_mask, verbose=False)
test_accuracy = logic.test_gnn(model, features, labels, edges, test_mask)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.7833
